In [ ]:
from pathlib import Path

import numpy as np
import torch
from cellpose import io, models, plot, utils
from matplotlib import pyplot as plt
from tifffile import imread, imwrite
from tqdm import tqdm

In [ ]:
# check gpu
use_gpu = torch.cuda.is_available()
print(use_gpu)

## Load Data

In [ ]:
data_dir = Path("../data")
data_folders = sorted([f for f in data_dir.iterdir() if f.is_dir()])

print("Found the following folders:")
for folder in data_folders:
    print(folder.stem)

## Check Channel

In [ ]:
example_img = list(data_folders[0].glob("*.png"))[0]
example_data = io.imread(example_img)
plt.imshow(example_data)

## Run Prediction

In [ ]:
# load my model
model = models.CellposeModel(
    gpu=True,
    pretrained_model="../models/Chronic_LSL_NeuN_Final",
)
diameter = model.diam_labels
chan = [2, 0]  # 2 for green

In [ ]:
output_dir = Path("../output")
output_dir.mkdir(exist_ok=True)

for folder in data_folders:
    print(f"Processing {folder.stem}")
    group_dir = output_dir / folder.stem
    group_dir.mkdir(exist_ok=True)

    preview_dir = group_dir / "preview"
    preview_dir.mkdir(exist_ok=True)
    mask_dir = group_dir / "mask"
    mask_dir.mkdir(exist_ok=True)

    img_list = sorted(folder.glob("*.png"))

    for img_path in tqdm(img_list):
        img = io.imread(img_path)
        masks, flows, styles = model.eval(img, diameter=diameter, channels=[2, 0])

        # save masks
        imwrite(str(mask_dir / img_path.stem) + "_masks.tif", masks, compression="zlib")

        # save an mask overlaid image
        plt.imshow(img)
        plt.imshow(masks != 0, alpha=0.5)
        plt.axis("off")
        plt.savefig(
            str(preview_dir / img_path.stem) + "_preview.png",
            dpi=600,
            bbox_inches="tight",
            pad_inches=0,
        )
        plt.close()
